# Markdown for Matching Fires from Multiple Years
-------------
C. Herbert, Date Started: 5.31.21

*Notes: This Markdown uses Python 3 and geemap package. If you do not have these downloaded, along with the packages required, you will need to do this before being able to run this workbook. To uncomment code, remove the '#' symbol preceeding the line of code.*

Prior to running this markdown you will need to be able to read the underlying Google Earth Engine (GEE) scripts to generate three continuous burn severity, Normalized Difference Moisture Index (NDMI), and Soil Adjusted Vegetation Index (SAVI). You can either run outputs yourself or connect to the public assets C. Herbert generated.

**Overview**
- Part I. (Python) Calls data from GEE to create CSVs of data sampling golf courses and non-golf courses that burned from 1984-2020
- Part II. (Python) Reads in the CSVs created in Part I and creates some exploratory graphs
- Part III. (R) Uses R package MatchIt to create propoensity scores + more graphs + match points
- Part IV. (R) Uses linear regression to create t-test of coefficients

In [1]:
# Import packages 
import ee
import geemap
import os
# import rpy2
ee.Initialize() # need to initialize the first time you use GEE
import ee, datetime
import pandas as pd
import numpy as np
from scipy.special import expit
# import seaborn as sns
# from matplotlib import pyplot as plt # duplicated with the following line
# import matplotlib.pyplot as plt
# from matplotlib import style
import warnings
warnings.filterwarnings('ignore')
# from pymatch.Matcher import Matcher

%matplotlib inline
import glob

## I. Define Boundaries and Create Sample Grid
read in shapefiles of fire and landscape feature (golf) from GEE and define sample area
### a. define the fire data

In [4]:
# imput the year of fire and the fire name
year_num = 2003
year = str(year_num)
fire_name = 'SIMI FIRE' # 'SIMI FIRE'  

### b. select golf course and fire for input

In [5]:
# downloaded fire perimeter data on 5.6.2021, convert gdb to shp in ArcPro 
# direct download link: https://frap.fire.ca.gov/media/3nrpp42r/fire20_1.zip 
golf = ee.FeatureCollection('users/claudiaherbert/GolfCourse_FiresOverlapQuar_20211013')
fires = ee.FeatureCollection('users/claudiaherbert/Firesp_20_1_update3').filter(ee.Filter.stringContains('YEAR_', year)).filter(ee.Filter.stringStartsWith('FIRE_NAME', fire_name))

# select the golf course relevant to an input fire
courses_year = golf.filter(ee.Filter.stringContains('FIRE_NAME', fire_name))

In [6]:
# need to select fires that overlap with burned treatment areas
filt = ee.Filter.intersects('.geo', courses_year.geometry())
fires_cntl = fires.filter(filt)
fire_diff = fires_cntl.geometry().difference(courses_year.geometry())
fire_poly = ee.FeatureCollection(fire_diff)

In [7]:
# creating the same because I have some datasets from the golf courses that seem to not be fully burned
filt_t = ee.Filter.intersects('.geo', fire_poly.geometry())
fires_treat = courses_year.filter(filt_t)
fire_diff_t = fires_cntl.geometry().difference(fire_poly.geometry())
fire_poly_treat = ee.FeatureCollection(fire_diff_t)

### c. create the sample grid
Make all of the zones into different samples that will be one mask

In [8]:
# this is where I need to make a loop that goes through each of the fires for 
# one year and pulls the area size to sample on density
courses_area = fire_poly_treat.map(lambda feature: ee.Feature(feature.set({'areaHa': feature.geometry().area().divide(100 * 100)})))
fires_area = fire_poly.map(lambda feature: ee.Feature(feature.set({'areaHa': feature.geometry().area().divide(100 * 100)})))

In [9]:
# Check that you are selecting the correct sized fire and golf course
# a zero for either indicates that you may have had an error in your previous sections
courses_samp = round(courses_area.aggregate_sum('areaHa').getInfo())
fire_samp = round(fires_area.aggregate_sum('areaHa').getInfo())

print('Total Golf Course Area:', courses_samp)
print('Total Fire Area:', fire_samp)

Total Golf Course Area: 160
Total Fire Area: 43382


In [10]:
# Golf Course
features = ee.FeatureCollection([ee.Feature(fires_area.geometry(), {'group': 0}), ee.Feature(courses_area.geometry(), {'group': 1})])
treatment = features.reduceToImage(['group'], ee.Reducer.mean()).select('mean').rename('group')

In [11]:
# this should be at 100 to make your sample point grid
points = treatment.select('group').sampleRegions(collection= features, scale= 100, geometries= True, properties= ['group'])

Check your work:

In [12]:
# # Uncomment the following to check that you called the intended fire and golf course
# Map = geemap.Map()
# naip_url = 'https://services.nationalmap.gov/arcgis/services/USGSNAIPImagery/ImageServer/WMSServer?'
# Map.add_wms_layer(url=naip_url, layers='0', name='NAIP Imagery', format='image/png', shown=True)
# Map.addLayer(fires_area, {'color': 'd63000'}, 'difference') 
# Map.addLayer(fires, {'color': 'd63000'}, 'testing 2') 
# Map.addLayer(courses_year, {'color': 'd63000'}, 'testing') 
# Map.addLayer(courses_area, {'color': 'ffea00', 'width': 10, 'lineType': 'solid',}, 'Treatment Area')
# Map

In [13]:
Map = geemap.Map()
naip_url = 'https://services.nationalmap.gov/arcgis/services/USGSNAIPImagery/ImageServer/WMSServer?'
Map.add_wms_layer(url=naip_url, layers='0', name='NAIP Imagery', format='image/png', shown=True)
Map.addLayer(treatment, {'bands': ['group'], 'palette': ['#62ff42', ' #fb0aff'], 'min': 0.0, 'max': 2, 'opacity': 1.0}, 'Combined Sample Area')
Map.addLayer(points, {'color': 'd63000'}, 'control points')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## II. Create the fire-specific data
Read in the layers from GEE needed for matching

In [14]:
# Precip: CHIRPS
dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')

# end date exclusive so range extends to following quarter
# this calculates the total rainfall in mm/day during these three month periods
precip_1 = dataset.filter(ee.Filter.date(year + '-01-01', year + '-04-01')).select('precipitation').sum()
precip_2 = dataset.filter(ee.Filter.date(year + '-04-01', year + '-07-01')).select('precipitation').sum()
precip_3 = dataset.filter(ee.Filter.date(year + '-07-01', year + '-10-01')).select('precipitation').sum()
precip_4 = dataset.filter(ee.Filter.date(year + '-10-01', year + '-12-31')).select('precipitation').sum()

In [15]:
# latitude + longitude
latitude = ee.Image.pixelLonLat().select('latitude')
longitude = ee.Image.pixelLonLat().select('longitude')

In [16]:
# Creating a unique identifer for each fire
fires_info = fires_cntl.getInfo()

ids = []

for i in range (0,len(fires_info['features'])): 
    fire_dict = fires_info['features'][i]
    fid = fire_dict['properties']['FIRE_NAME']  #['GIS_ACRES']
    s_fid = str(fid)
    nospace = s_fid.replace(" ", "")
    ids.append(nospace)
    
uid = np.unique(ids)
print(uid)

['SIMIFIRE']


In [17]:
fire_id = fires_cntl.map(lambda feature: ee.Feature(feature.set({'FireId': ee.Number.parse(feature.get('id'))})))

The following fire severity data should be public if you are running one of the fires in our paper. If you are running a different fire, you will need to produce these outputs by going to our GEE script. Chaneg path as needed. 

In [18]:
# Reading in the continuous data

# create an empty list for us to add names
rdnbr_paths = []
# creating path names for rdnbr
for i in range(0,len(uid)):   
    path = ee.Image('users/claudiaherbert/InitialBurnSev_v2/' + str(uid[i]) + '_rdnbr_w_offset_' + year)
    rdnbr_paths.append(path)

rdnbr_w_offset = ee.ImageCollection(rdnbr_paths).max()
print("Finished with rdnbr_w_offset")

# create an empty list for us to add names
dnbr_paths = []
# creating path names for dnbr
for i in range(0,len(uid)):   
    path = ee.Image('users/claudiaherbert/InitialBurnSev_v2/' + str(uid[i]) + '_dnbr_w_offset_' + year)
    dnbr_paths.append(path)
    
dnbr_w_offset = ee.ImageCollection(dnbr_paths).max()
print("Finished with dnbr_w_offset")
    
# create an empty list for us to add names
rbr_paths = []
# creating path names for rbr
for i in range(0,len(uid)):   
    path = ee.Image('users/claudiaherbert/InitialBurnSev_v2/' + str(uid[i]) + '_rbr_w_offset_' + year)
    rbr_paths.append(path)

rbr_w_offset = ee.ImageCollection(rbr_paths).max()
print("Finished with rdnbr_w_offset")

Finished with rdnbr_w_offset
Finished with dnbr_w_offset
Finished with rdnbr_w_offset


In [19]:
# Reading in the veg NDMI data

ndmi_1_paths = []
# creating path names for ndmi
for i in range(0,len(uid)):   
    path = ee.Image('users/claudiaherbert/NDMI_v2/' + str(uid[i]) + '_NDMI_1_' + year)
    ndmi_1_paths.append(path)

ndmi_1 = ee.ImageCollection(ndmi_1_paths).max()
print("Finished with NDMI paths")

# create an empty list for us to add names
# users/claudiaherbert/NDMI/TAYLOR_NDMI_3_1998
ndmi_3_paths = []
# creating path names for ndmi
for i in range(0,len(uid)):   
    path = ee.Image('users/claudiaherbert/NDMI_v2/' + str(uid[i]) + '_NDMI_3_' + year)
    ndmi_3_paths.append(path)

ndmi_3 = ee.ImageCollection(ndmi_3_paths).max()
print("Finished with NDMI paths")

ndmi_6_paths = []
# creating path names for ndmi
for i in range(0,len(uid)):   
    path = ee.Image('users/claudiaherbert/NDMI_v2/' + str(uid[i]) + '_NDMI_6_' + year)
    ndmi_6_paths.append(path)

ndmi_6 = ee.ImageCollection(ndmi_6_paths).max()
print("Finished with NDMI paths")


Finished with NDMI paths
Finished with NDMI paths
Finished with NDMI paths


In [20]:
savi_paths = []
# creating path names for savi
for i in range(0,len(uid)):   
    path = ee.Image('users/claudiaherbert/SAVI_v2/' + str(uid[i]) + '_SAVI_' + year)
    savi_paths.append(path)

savi = ee.ImageCollection(savi_paths).max()
print("Finished with SAVI paths")

Finished with SAVI paths


In [21]:
# # Getting a fire ID to use to keep analysis inter-fire
# # fires_name = ee.FeatureCollection('users/claudiaherbert/FRAP_1kmGC_Names_2').filter(ee.Filter.eq('Year', year_num))

def getCentroid(feature): 
    keepProperties = ['GIS_ACRES']
    centroid = feature.geometry()
    return ee.Feature(centroid).copyProperties(feature, keepProperties);

centroids = fires_cntl.map(getCentroid)

landAreaImg = centroids.filter(ee.Filter.notNull(['GIS_ACRES'])).reduceToImage(
    properties = ['GIS_ACRES'],
    reducer = ee.Reducer.first()).rename('GIS_ACRES')

In [22]:
# Load SRTM
srtm = ee.Image('USGS/SRTMGL1_003')

# Calculate elevation 
elevation = srtm.select('elevation')
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)

In [23]:
# Landcover pre-fire
landcoverd_ds = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')

# NLCD Epochs 1992, 2001, 2004, 2006, 2008, 2011, 2013 and 2016
# not using 1992 because there were different methods to determine landcover
# I choose to use the epoch prior to the fire (where available) to get match on landcover pre-fire

if year_num >= 2016: 
    # NLCD 2016
    nlcd2016 = landcoverd_ds.filter(ee.Filter.eq('system:index', '2016')).first()
    landcover = nlcd2016.select('landcover')
    print("Using NLCD 2016")
elif year_num >= 2013: # or year_num < 2016:
    # NLCD 2013
    nlcd2013 = landcoverd_ds.filter(ee.Filter.eq('system:index', '2013')).first()
    landcover = nlcd2013.select('landcover')
    print("Using NLCD 2013")
elif year_num >= 2011: # or year_num < 2013:
    # NLCD 2011
    nlcd2011 = landcoverd_ds.filter(ee.Filter.eq('system:index', '2011')).first()
    landcover = nlcd2011.select('landcover')
    print("Using NLCD 2011")
elif year_num >= 2008: # or year_num < 2011:
    # NLCD 2008
    nlcd2008 = landcoverd_ds.filter(ee.Filter.eq('system:index', '2008')).first()
    landcover = nlcd2008.select('landcover')
    print("Using NLCD 2008")    
elif year_num >= 2006: # or year_num < 2008: 
    # NLCD 2006
    nlcd2006 = landcoverd_ds.filter(ee.Filter.eq('system:index', '2006')).first()
    landcover = nlcd2006.select('landcover')
    print("Using NLCD 2006")
elif year_num >= 2004: # or year_num < 2006:
    # NLCD 2004
    nlcd2004 = landcoverd_ds.filter(ee.Filter.eq('system:index', '2004')).first()
    landcover = nlcd2004.select('landcover')
    print("Using NLCD 2004")
else:     
    # NLCD 2001
    nlcd2001 = landcoverd_ds.filter(ee.Filter.eq('system:index', '2001')).first()
    landcover = nlcd2001.select('landcover')
    print("Using NLCD 2001")

Using NLCD 2001


In [24]:
# Census
# Combination of ACS and Decennial Census accessed via 'tidycensus' package in R

if year_num >= 2014: 
    # ACS 2014-2018
    acs_2018 = ee.FeatureCollection('users/claudiaherbert/ca_18_20220106_v01').select('estimate')
    income = acs_2018.reduceToImage(['estimate'], ee.Reducer.mean()).select('mean').rename('income')
    print("Using ACS 2014 - 2018")
elif year_num >= 2010: # or year_num < 2016:
    # ACS 2011 - 2013
    acs_2013 = ee.FeatureCollection('users/claudiaherbert/ca_13_20220106_v01').select('estimate')
    income = acs_2013.reduceToImage(['estimate'], ee.Reducer.mean()).select('mean').rename('income')
    print("Using ACS 2011 - 2013")
elif year_num >= 2005: # or year_num < 2013:
    # ACS 2005 - 2009
    acs_2009 = ee.FeatureCollection('users/claudiaherbert/ca_09_20220106_v01').select('estimate')
    income = acs_2009.reduceToImage(['estimate'], ee.Reducer.mean()).select('mean').rename('income')
    print("Using ACS 2005 - 2009")
else:     
    # Decennial 2000
    acs_2000 = ee.FeatureCollection('users/claudiaherbert/ca_00_20220106_v01').select('value')
    income = acs_2000.reduceToImage(['value'], ee.Reducer.mean()).select('mean').rename('income')
    print("Using Decennial 2000")

Using Decennial 2000


MTBS may not be available for your fire--check availability

In [25]:
mtbs_one = ee.Image('users/claudiaherbert/MTBS/mtbs_CA_' + year).select('b1')

### Make a raster stack

In [26]:
# creating a combined image with all relevant stats
# dropped the following to test group

combined_img = elevation.addBands(slope).addBands(aspect).addBands(landcover) \
            .addBands(precip_1).addBands(precip_2).addBands(precip_3).addBands(precip_4) \
            .addBands(latitude).addBands(longitude).addBands(landAreaImg).addBands(dnbr_w_offset) \
            .addBands(rbr_w_offset).addBands(rdnbr_w_offset).addBands(ndmi_3).addBands(ndmi_6) \
            .addBands(savi).addBands(treatment).addBands(income)

Check your work

In [27]:
Map = geemap.Map()
Map.addLayer(combined_img, {'color': 'fd0808e2'}, 'Images')
Map.addLayer(points, {'color': 'd63000'}, 'control points')
# Map.addLayer(mtbs_one, {}, 'mtbs') # determine whether MTBS is available for this fire
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## III. Extract points using geemap

In [28]:
 # Change this to a local path you want data downloaded
work_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_csv = os.path.join(work_dir, 'grid_point_100m_' + fire_name + "_"+ year + '.csv')
geemap.extract_values_to_points(points, combined_img, out_csv)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\claud\Downloads\grid_point_100m_SIMI FIRE_2003.csv


## IV. Read in CSVs and produce a single output
After you have run the above workflow for all of the fires, you can combine the CSV outputs to be a single CSV that we will pass the the R Markdown for PSM and regression analysis

In [ ]:
import glob
list_files = glob.glob(" << insert the path to where you downloaded your files >> /*.csv")
print(glob.glob("<< insert the path to where you downloaded your files >>/*.csv"))

In [ ]:
df = pd.DataFrame()

for x in range(0,len(list_files)):
    df_1 = pd.read_csv(list_files[x])
    year = int(list_files[x][-8:-4]) # will need to change this index if you change where the file names
    df_1["year"] = year
    df = df.append(df_1, ignore_index = False)
print("Finished with reading in CSVs")

In [ ]:
df.to_csv('<< insert the path to where you downloaded your files >> /GolfCourseSamplesMatching_CH_20220107_v01.csv', encoding='utf-8')

In [ ]:
# optional, check the df
df